In [1]:
import torch
import pathpyG as pp
from collections import defaultdict

from torch_geometric.utils import cumsum, coalesce, degree, sort_edge_index
from torch_geometric.data import Data

from pathpyG import Graph, TemporalGraph, MultiOrderModel

pp.config['torch']['device'] = 'cuda'

In [2]:
def long_temporal_graph() -> TemporalGraph:
    """Return a temporal graph with 20 time-stamped edges."""
    tedges = [('a', 'b', 1), ('b', 'c', 5), ('c', 'd', 9), ('c', 'e', 9),
              ('c', 'f', 11), ('f', 'a', 13), ('a', 'g', 18), ('b', 'f', 21),
              ('a', 'g', 26), ('c', 'f', 27), ('h', 'f', 27), ('g', 'h', 28),
              ('a', 'c', 30), ('a', 'b', 31), ('c', 'h', 32), ('f', 'h', 33),
              ('b', 'i', 42), ('i', 'b', 42), ('c', 'i', 47), ('h', 'i', 50)]
    return TemporalGraph.from_edge_list(tedges)
t = long_temporal_graph()

In [3]:
pp.algorithms.centrality.temporal_closeness_centrality(t, delta=5)

k = 3, edge_index size = 7
k = 4, edge_index size = 3
tensor([[inf, 1., 1., 3., 3., inf, 1., 2., inf],
        [inf, inf, 1., 2., 2., 1., inf, inf, 1.],
        [2., inf, inf, 1., 1., 1., 3., 1., 1.],
        [inf, inf, inf, inf, inf, inf, inf, inf, inf],
        [inf, inf, inf, inf, inf, inf, inf, inf, inf],
        [1., inf, inf, inf, inf, inf, 2., 1., inf],
        [inf, inf, inf, inf, inf, inf, inf, 1., inf],
        [inf, inf, inf, inf, inf, 1., inf, inf, 1.],
        [inf, 1., inf, inf, inf, inf, inf, inf, inf]], device='cuda:0')
v: a, idx: 0
tensor([0., 0., 4., 0., 0., 8., 0., 0., 0.], device='cuda:0')
v: b, idx: 1
tensor([8., 0., 0., 0., 0., 0., 0., 0., 8.], device='cuda:0')
v: c, idx: 2
tensor([8., 8., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
v: d, idx: 3
tensor([2.6667, 4.0000, 8.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')
v: e, idx: 4
tensor([2.6667, 4.0000, 8.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0

defaultdict(<function pathpyG.algorithms.centrality.temporal_closeness_centrality.<locals>.<lambda>()>,
            {'a': 12.0,
             'b': 16.0,
             'c': 16.0,
             'd': 14.666666984558105,
             'e': 14.666666984558105,
             'f': 24.0,
             'g': 14.666666984558105,
             'h': 28.0,
             'i': 24.0})

In [90]:
tedges = [
    ("a", "b", 1),
    ("a", "b", 2),
    ("a", "c", 2),
    ("b", "a", 3),
    ("b", "c", 3),
    ("d", "c", 4),
    ("a", "b", 4),
    ("c", "b", 4),
    ("c", "d", 5),
    ("b", "e", 5),
    ("c", "e", 3),
    ("c", "b", 6),
]
t = pp.TemporalGraph.from_edge_list(tedges)
print(t.mapping)
print(t.N)
print(t.M)

a -> 0
b -> 1
c -> 2
d -> 3
e -> 4

5
12


In [89]:
t = pp.TemporalGraph.from_csv('../data/ants_1_1.tedges')
print(t)

Temporal Graph with 89 nodes, 947 unique edges and 1911 events in [0.0, 1438.0]

Graph attributes
	t		<class 'torch.Tensor'> -> torch.Size([1911])
	src		<class 'torch.Tensor'> -> torch.Size([1911])
	dst		<class 'torch.Tensor'> -> torch.Size([1911])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'t', 'src', 'dst'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [91]:
m = pp.MultiOrderModel.from_temporal_graph(t, max_order=4, delta=2)
print(m.layers[2])
for i, e in enumerate(m.layers[2].edges):
    print(e, m.layers[2].data.edge_weight[i])

Directed graph with 9 nodes and 11 edges

Node attributes
	node_sequence		<class 'torch.Tensor'> -> torch.Size([9, 2])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([11])

Graph attributes
	num_nodes		<class 'int'>

(('a', 'b'), ('b', 'e')) tensor(2., device='cuda:0')
(('a', 'b'), ('b', 'c')) tensor(2., device='cuda:0')
(('a', 'b'), ('b', 'a')) tensor(1., device='cuda:0')
(('a', 'c'), ('c', 'e')) tensor(1., device='cuda:0')
(('a', 'c'), ('c', 'b')) tensor(1., device='cuda:0')
(('b', 'a'), ('a', 'b')) tensor(1., device='cuda:0')
(('b', 'c'), ('c', 'd')) tensor(1., device='cuda:0')
(('b', 'c'), ('c', 'b')) tensor(1., device='cuda:0')
(('c', 'b'), ('b', 'e')) tensor(1., device='cuda:0')
(('d', 'c'), ('c', 'd')) tensor(1., device='cuda:0')
(('d', 'c'), ('c', 'b')) tensor(1., device='cuda:0')


In [76]:
def temporal_shortest_paths(g: pp.TemporalGraph, delta, return_counts=True) -> tuple[torch.Tensor, dict[int, torch.Tensor]]:
    """
    Calculates all shortest paths between all pairs of nodes
    based on a set of empirically observed paths.
    """
    shortest_paths = {}
    shortest_path_lengths = torch.full((g.N, g.N), float("inf"), device=g.data.edge_index.device)
    shortest_path_lengths.fill_diagonal_(float(0))
    shortest_path_counts = torch.full((g.N, g.N), 0, device=g.data.edge_index.device)
    out_degree = degree(g.data.edge_index[0], num_nodes=g.N)
    in_degree = degree(g.data.edge_index[1], num_nodes=g.N)

    # Set combinations where a path is impossible to -1 to improve stopping conditions
    shortest_path_lengths[out_degree == 0] = -1
    shortest_path_lengths[:, in_degree == 0] = -1

    # first-order edge index
    edge_index, timestamps = sort_edge_index(g.data.edge_index, g.data.t)
    node_sequence = torch.arange(g.data.num_nodes, device=edge_index.device).unsqueeze(1)
    
    # second-order edge index with time-respective filtering
    k = 2
    null_model_edge_index = pp.MultiOrderModel.lift_order_edge_index(
        edge_index, num_nodes=node_sequence.size(0)
    )
    # Update node sequences
    node_sequence = torch.cat([node_sequence[edge_index[0]], node_sequence[edge_index[1]][:, -1:]], dim=1)
    # Remove non-time-respecting higher-order edges
    time_diff = timestamps[null_model_edge_index[1]] - timestamps[null_model_edge_index[0]]
    non_negative_mask = time_diff > 0
    delta_mask = time_diff <= delta
    time_respecting_mask = non_negative_mask & delta_mask
    edge_index = null_model_edge_index[:, time_respecting_mask]

    # Use node sequences to update shortest path lengths
    def update_paths(node_sequence, k):
        path_starts = node_sequence[:, 0]
        path_ends = node_sequence[:, -1]
        mask = shortest_path_lengths[path_starts, path_ends] >= k - 1
        shortest_path_lengths[path_starts[mask], path_ends[mask]] = k - 1
        if mask.sum() > 0:
            shortest_paths[k-1] = torch.unique(node_sequence[mask], dim=0)
        if return_counts:
            pairs = torch.cat([shortest_paths[k-1][:, 0].unsqueeze(1), shortest_paths[k-1][:, -1].unsqueeze(1)], dim=1)
            unique_pairs, counts = torch.unique(pairs, dim=0, return_counts=True)
            shortest_path_counts[unique_pairs[:, 0], unique_pairs[:, 1]] = counts
    update_paths(node_sequence, k)

    k = 3
    while torch.max(shortest_path_lengths) > k and edge_index.size(1) > 0:
        print(f"k = {k}, edge_index size = {edge_index.size(1)}")
        ho_index = MultiOrderModel.lift_order_edge_index(edge_index, num_nodes=node_sequence.size(0))
        node_sequence = torch.cat([node_sequence[edge_index[0]], node_sequence[edge_index[1]][:, -1:]], dim=1)
        edge_index = ho_index
        print("\tOrder lifted.")
        update_paths(node_sequence, k)
        print("\tPaths updated.")
        k += 1

    shortest_path_lengths[shortest_path_lengths == -1] = float("inf")
    if return_counts:
        return shortest_paths, shortest_path_lengths, shortest_path_counts
    return shortest_paths, shortest_path_lengths

In [92]:
sp, sp_lengths, counts = temporal_shortest_paths(t, delta=1)
# print(sp_lengths)
# print(sp)

k = 3, edge_index size = 8
	Order lifted.
	Paths updated.
k = 4, edge_index size = 4
	Order lifted.
	Paths updated.
k = 5, edge_index size = 2
	Order lifted.
	Paths updated.


In [78]:
counts

tensor([[0, 1, 1, 0, 1],
        [1, 0, 1, 0, 1],
        [0, 1, 0, 1, 1],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0]], device='cuda:0')

In [79]:
sp_lengths

tensor([[0., 1., 2., inf, 2.],
        [1., 0., 1., inf, 1.],
        [inf, 1., 0., 1., 2.],
        [inf, inf, 1., 0., inf],
        [inf, inf, inf, inf, inf]], device='cuda:0')

In [80]:
sp

{1: tensor([[0, 1],
         [1, 0],
         [1, 2],
         [1, 4],
         [2, 1],
         [2, 3],
         [3, 2]], device='cuda:0'),
 2: tensor([[0, 1, 2],
         [0, 1, 4],
         [2, 1, 4]], device='cuda:0'),
 3: tensor([], device='cuda:0', size=(0, 4), dtype=torch.int64),
 4: tensor([], device='cuda:0', size=(0, 5), dtype=torch.int64)}

In [85]:
def temporal_betweenness_centrality(g: TemporalGraph, delta, normalized=False):
    """Calculates the betweenness of nodes based on observed shortest paths
    between all pairs of nodes

    Parameters
    ----------
    paths:
        Paths object
    normalized: bool
        normalize such that largest value is 1.0

    Returns
    -------
    dict
    """
    # assert isinstance(paths, pp.PathData), "argument must be an instance of pathpy.Paths"
    node_centralities = torch.zeros(g.N, device=g.data.edge_index.device)

    # Log.add('Calculating betweenness in paths ...', Severity.INFO)

    sp, _, counts = temporal_shortest_paths(g, delta, return_counts=True)

    for v in range(g.N):
        for paths in list(sp.values())[1:]:
            v_as_start = paths[:, 0] == v
            v_as_end = paths[:, -1] == v
            paths_not_v = paths[~(v_as_start | v_as_end)]
            mask = paths_not_v == v
            fractions = (mask.sum(dim=1) > 0) / counts[paths_not_v[:, 0], paths_not_v[:, -1]]
            node_centralities[v] += fractions.sum()

    if normalized:
        max_centr = max(node_centralities.values())
        for v in node_centralities:
            node_centralities[v] /= max_centr

    # Log.add('finished.')
    return node_centralities

In [102]:
pp.algorithms.centrality.temporal_betweenness_centrality(t, delta=5)

Directed graph with 17 nodes and 7 edges

Graph attributes
	num_nodes		<class 'int'>

Processing root 1/13
Processing root 11/13


defaultdict(<function pathpyG.algorithms.centrality.temporal_betweenness_centrality.<locals>.<lambda>()>,
            {'b': 2.0,
             'c': 2.5,
             'g': 0.5,
             'f': 1.0,
             'a': 1.0,
             'd': 0,
             'e': 0,
             'h': 0,
             'i': 0})

In [94]:
sp

{1: tensor([[0, 1],
         [0, 2],
         [1, 0],
         [1, 2],
         [1, 4],
         [2, 1],
         [2, 3],
         [2, 4],
         [3, 2]], device='cuda:0'),
 2: tensor([[0, 1, 4],
         [0, 2, 4]], device='cuda:0'),
 3: tensor([], device='cuda:0', size=(0, 4), dtype=torch.int64),
 4: tensor([], device='cuda:0', size=(0, 5), dtype=torch.int64)}

In [17]:
sp_lengths[sp_lengths < 100].max()

tensor(31., device='cuda:0')

In [7]:
pp.algorithms.centrality.temporal_closeness_centrality(t, delta=5, normalized=False)

Directed graph with 1910 nodes and 562 edges

Graph attributes
	num_nodes		<class 'int'>

Processing root 1/1417
Processing root 11/1417
Processing root 21/1417
Processing root 31/1417
Processing root 41/1417
Processing root 51/1417
Processing root 61/1417
Processing root 71/1417
Processing root 81/1417
Processing root 91/1417
Processing root 101/1417
Processing root 111/1417
Processing root 121/1417
Processing root 131/1417
Processing root 141/1417
Processing root 151/1417
Processing root 161/1417
Processing root 171/1417
Processing root 181/1417
Processing root 191/1417
Processing root 201/1417
Processing root 211/1417
Processing root 221/1417
Processing root 231/1417
Processing root 241/1417
Processing root 251/1417
Processing root 261/1417
Processing root 271/1417
Processing root 281/1417
Processing root 291/1417
Processing root 301/1417
Processing root 311/1417
Processing root 321/1417
Processing root 331/1417
Processing root 341/1417
Processing root 351/1417
Processing root 361/1

defaultdict(<function pathpyG.algorithms.centrality.temporal_closeness_centrality.<locals>.<lambda>()>,
            {'Y_WY': 14.2,
             'WRBB': 15.916666666666666,
             'WRR_': 5.0,
             'GGRR': 19.642857142857142,
             'WG_R': 15.0,
             'YWGW': 10.333333333333334,
             'YYGG': 12.5,
             'GBGR': 5.0,
             'GRWG': 12.666666666666666,
             'YY_W': 6.0,
             'G_W_': 6.0,
             'G___small': 2.0,
             'YW__': 2.0,
             'YYGGmid': 19.833333333333332,
             '____brood': 6.25,
             'GGWY': 17.0,
             'G_R_': 2.0,
             '____topleft': 11.0,
             'G_GW': 10.5,
             'GYYY': 4.0,
             'WBGW': 8.833333333333332,
             'Y__W': 4.5,
             'YYYY': 3.5,
             'GWRG': 10.0,
             'YGWW': 16.916666666666668,
             'WRWR': 14.333333333333334,
             'G___big': 2.0,
             'GY__': 3.3333333333333335,
   

In [8]:
pp.algorithms.centrality.temporal_betweenness_centrality(t, delta=5, normalized=False)

Directed graph with 1910 nodes and 562 edges

Graph attributes
	num_nodes		<class 'int'>

Processing root 1/1417
Processing root 11/1417
Processing root 21/1417
Processing root 31/1417
Processing root 41/1417
Processing root 51/1417
Processing root 61/1417
Processing root 71/1417
Processing root 81/1417
Processing root 91/1417
Processing root 101/1417
Processing root 111/1417
Processing root 121/1417
Processing root 131/1417
Processing root 141/1417
Processing root 151/1417
Processing root 161/1417
Processing root 171/1417
Processing root 181/1417
Processing root 191/1417
Processing root 201/1417
Processing root 211/1417
Processing root 221/1417
Processing root 231/1417
Processing root 241/1417
Processing root 251/1417
Processing root 261/1417
Processing root 271/1417
Processing root 281/1417
Processing root 291/1417
Processing root 301/1417
Processing root 311/1417
Processing root 321/1417
Processing root 331/1417
Processing root 341/1417
Processing root 351/1417
Processing root 361/1

defaultdict(<function pathpyG.algorithms.centrality.temporal_betweenness_centrality.<locals>.<lambda>()>,
            {'WG_R': 4.0,
             'GR_Y2': 9.0,
             'GRWG': 5.0,
             'GGRR': 5.0,
             'Y_WY': 4.0,
             'YYGGmid': 4.0,
             'WRWR': 1.0,
             'GRYY': 1.0,
             'WBGG': 4.0,
             '____brood': 2.5,
             'GGWY': 4.0,
             'GR__': 3.0,
             'G_R_': 1.0,
             'GWRG': 3.0,
             'WGWB': 8.0,
             '____almost': 2.0,
             'Q': 7.0,
             '_WYW': 2.0,
             'RWWG': 5.0,
             '_W_Y': 5.0,
             'YWW_': 4.0,
             'WRR_': 1.0,
             'YWGW': 1.0,
             'G_GW': 2.0,
             '____topleft': 2.0,
             'WYGG': 4.0,
             'RWGY': 2.0,
             '_R__': 7.0,
             'YYRB': 2.0,
             'GBGR': 1.0,
             '_WWY': 4.0,
             'GGWW': 1.5,
             'YYGG': 3.0,
             'YY_